In [ ]:
# !pip3 install numpy
# !pip3 install pandas
# !pip3 install csv
# !pip3 install openpyxl

In [1]:
# 1 Load the Data

import pandas as pd
import os
import re

north_path = r"datasets\พจนานุกรมภาษาถิ่นเหนือ.xlsx"
isan_path = r"datasets\พจนานุกรมภาษาถิ่นอีสาน.xlsx"
south_path = r"datasets\พจนานุกรมภาษาถิ่นใต้.xlsx"
orst_path = r"datasets\ราชบัณฑิตฯ"

def load_excel_sheets(file_path, exclude_sheets):
    all_sheets = pd.ExcelFile(file_path).sheet_names
    importing_sheets = [sheet for sheet in all_sheets if sheet not in exclude_sheets]
    return pd.read_excel(file_path, sheet_name=importing_sheets, header=None)

north = load_excel_sheets(north_path, ['อักษรย่อชนิดคำ', 'Example', 'note'])
isan = pd.read_excel(isan_path, sheet_name=None, header=None)
south = pd.read_excel(south_path, sheet_name=None, header=None)

def load_excel_files(directory):
    excel_files = {}
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            name = os.path.splitext(filename)[0]
            excel_files[name] = pd.read_excel(os.path.join(directory, filename))
    return excel_files

orst = load_excel_files(orst_path)

In [ ]:
# 2 Inspect the Data

# print(north["ก"].head(n=2))
# print(south["ก"].head(n=2))
# print(isan["ก"].head(n=2))

for key, value in orst.items():
  print(f"``{key}\n{orst[key].head(n=2)}\n")

In [ ]:
# 3 



In [ ]:
# 3 Skip Header-like Rows and Align Columns

pos_pattern = r"(น\.|ก\.|ว\.|สัน\.|สำนวน\.|ลัก\.|ล\.)"
alt_forms_keywords = ["ก็ว่า", "ก็เรียก"]

# def clean_alternative_forms(definition):
#     if "ก็ว่า" in definition and not definition.endswith("ก็ว่า."):
#         definition = definition.replace("ก็ว่า", "ก็ว่า.")
#     if "ก็เรียก" in definition and not definition.endswith("ก็เรียก."):
#         definition = definition.replace("ก็เรียก", "ก็เรียก.")
    
#     definition = re.sub(r"ก็ว่า\s*=\s*", "ก็ว่า. ", definition)
#     definition = re.sub(r"ก็เรียก,\s*บางถิ่นเรียก\s*", "ก็เรียก. ", definition)
    
#     return definition.strip()

def clean_alternative_forms(definition):
    definition = re.sub(r'\S+\s+(ก็ว่า)', r'\1', definition)
    definition = re.sub(r'\S+\s+(ก็เรียก)', r'\1', definition)
    definition = re.sub(r'ก็ว่า(?!\.)', 'ก็ว่า.', definition)
    definition = re.sub(r'ก็เรียก(?!\.)', 'ก็เรียก.', definition)
    definition = re.sub(r"ก็ว่า\s*=\s*", "ก็ว่า. ", definition)
    definition = re.sub(r"ก็เรียก,\s*บางถิ่นเรียก\s*", "ก็เรียก. ", definition)
    return definition.strip()

def align_columns_if_needed(row):
    if pd.notnull(row[0]) and "ช่อง" in str(row[0]):
        return None
    if pd.isnull(row[0]) and pd.notnull(row[1]):
        row[0] = None
    return row

def align_and_split_pos(df):
    new_rows = []
    current_headword = None
    
    for index, row in df.iterrows():
        row = align_columns_if_needed(row)
        if row is None:
            continue
        
        if pd.notnull(row[0]):
            current_headword = row[0]

        headword = current_headword
        
        if pd.notnull(row[1]):
            split_pos = re.split(pos_pattern, str(row[1]))
            
            if len(split_pos) > 1:
                for i in range(1, len(split_pos), 2):
                    pos = split_pos[i].strip()
                    definition = split_pos[i+1].strip() if i+1 < len(split_pos) else ''
                    new_rows.append([headword, pos, definition])
            else:
                new_rows.append([headword, row[1], row[2] if pd.notnull(row[2]) else ''])
        else:
            new_rows.append([headword, '', ''])
    
    final_rows = []
    for i in range(len(new_rows)):
        headword, pos, definition = new_rows[i]

        if any(keyword in str(definition) for keyword in alt_forms_keywords):
            if len(final_rows) > 0:
                definition = clean_alternative_forms(definition)
                if pd.notnull(final_rows[-1][2]):
                    final_rows[-1][2] += f", {definition.strip()}"
                else:
                    final_rows[-1][2] = f"{definition.strip()}"
        else:
            final_rows.append([headword, pos, definition])

    return pd.DataFrame(final_rows, columns=['Headword', 'POS', 'Definition'])

# Refactored Code
def process_and_save_dfs(data_dict, region, output_path):
    for sheet_name, df in data_dict.items():
        cleaned_df = align_and_split_pos(df)
        cleaned_df.to_csv(f"{output_path}/cleaned_{region}_{sheet_name}.csv", index=False, encoding='utf-8-sig')

output_path = r"output\csv_test1"

process_and_save_dfs(north, 'north', output_path)
process_and_save_dfs(isan, 'isan', output_path)
process_and_save_dfs(south, 'south', output_path)

In [ ]:
import pandas as pd
import re

# Define POS pattern and keywords for alternative forms
pos_pattern = r"(น\.|ก\.|ว\.|สัน\.|สำนวน\.|ลัก\.|ล\.)"
alt_forms_keywords = ["ก็ว่า", "ก็เรียก"]

# Helper function to clean alternative forms from definitions
def clean_alternative_forms(definition):
    # Standardize 'ก็ว่า' and 'ก็เรียก' formatting
    definition = re.sub(r"ก็ว่า\s*=\s*", "ก็ว่า. ", definition)
    definition = re.sub(r"ก็เรียก,\s*บางถิ่นเรียก\s*", "ก็เรียก. ", definition)
    return definition.strip()

# Main function to parse and format dictionary data
def align_and_split_pos(df):
    new_rows = []  # Store the parsed rows
    current_headword = None
    
    for index, row in df.iterrows():
        # Skip headers or invalid rows
        if pd.notnull(row[0]) and any(keyword in str(row[0]) for keyword in ["แม่คำ", "ช่อง"]):
            continue
        
        # Set the headword if available in the first column
        if pd.notnull(row[0]):
            current_headword = str(row[0])  # Ensure headword is a string
        
        # Ensure POS and Definition columns are strings
        pos_column = str(row[1]) if pd.notnull(row[1]) else ""
        definition_column = str(row[2]) if pd.notnull(row[2]) else ""
        
        # Handle POS and Definition
        if pos_column:
            split_pos = re.split(pos_pattern, pos_column)
            
            # If multiple POS/definition pairs, handle each separately
            if len(split_pos) > 1:
                for i in range(1, len(split_pos), 2):
                    pos = split_pos[i].strip()
                    definition = split_pos[i + 1].strip() if i + 1 < len(split_pos) else ""
                    
                    # Clean and add the row if definition is meaningful
                    definition = clean_alternative_forms(definition)
                    if definition:
                        new_rows.append([current_headword, pos, definition])
            else:
                pos = pos_column.strip()
                definition = clean_alternative_forms(definition_column)
                
                # Clean and add the single POS/definition entry
                if definition:
                    new_rows.append([current_headword, pos, definition])
    
    return pd.DataFrame(new_rows, columns=['Headword', 'POS', 'Definition'])

north = {sheet_name: align_and_split_pos(df) for sheet_name, df in north.items()}
isan = {sheet_name: align_and_split_pos(df) for sheet_name, df in isan.items()}
south = {sheet_name: align_and_split_pos(df) for sheet_name, df in south.items()}

output_path = r"output\csv_test2"

for sheet_name, df in north.items():
    df.to_csv(f"{output_path}\\cleaned_north_{sheet_name}.csv", index=False, encoding='utf-8-sig')

for sheet_name, df in isan.items():
    df.to_csv(f"{output_path}\\cleaned_isan_{sheet_name}.csv", index=False, encoding='utf-8-sig')

for sheet_name, df in south.items():
    df.to_csv(f"{output_path}\\cleaned_south_{sheet_name}.csv", index=False, encoding='utf-8-sig')


In [ ]:
# original

pos_pattern = r"(น\.|ก\.|ว\.|สัน\.|สำนวน\.|ลัก\.|ล\.)"
alt_forms_keywords = ["ก็ว่า", "ก็เรียก"]

def clean_alternative_forms(definition):
    if "ก็ว่า" in definition and not definition.endswith("ก็ว่า."):
        definition = definition.replace("ก็ว่า", "ก็ว่า.")
    if "ก็เรียก" in definition and not definition.endswith("ก็เรียก."):
        definition = definition.replace("ก็เรียก", "ก็เรียก.")
    
    definition = re.sub(r"ก็ว่า\s*=\s*", "ก็ว่า. ", definition)
    definition = re.sub(r"ก็เรียก,\s*บางถิ่นเรียก\s*", "ก็เรียก. ", definition)
    
    return definition.strip()

def align_columns_if_needed(row):
    if pd.notnull(row[0]) and "ช่อง" in str(row[0]):
        return None
    
    if pd.isnull(row[0]) and pd.notnull(row[1]):
        row[0] = None
    return row

def align_and_split_pos(df):
    new_rows = []
    current_headword = None
    
    for index, row in df.iterrows():
        row = align_columns_if_needed(row)
        if row is None:
            continue
        
        if pd.notnull(row[0]):
            current_headword = row[0]
        
        headword = current_headword
        
        if pd.notnull(row[1]):
            split_pos = re.split(pos_pattern, str(row[1]))
            
            if len(split_pos) > 1:
                for i in range(1, len(split_pos), 2):
                    pos = split_pos[i].strip()
                    definition = split_pos[i+1].strip() if i+1 < len(split_pos) else ''
                    new_rows.append([headword, pos, definition])
            else:
                new_rows.append([headword, row[1], row[2] if pd.notnull(row[2]) else ''])
        else:
            new_rows.append([headword, '', ''])
    
    final_rows = []
    for i in range(len(new_rows)):
        headword, pos, definition = new_rows[i]

        if any(keyword in str(definition) for keyword in alt_forms_keywords):
            if len(final_rows) > 0:
                definition = clean_alternative_forms(definition)
                if pd.notnull(final_rows[-1][2]):
                    final_rows[-1][2] += f", {definition.strip()}"
                else:
                    final_rows[-1][2] = f"{definition.strip()}"
        else:
            final_rows.append([headword, pos, definition])

    return pd.DataFrame(final_rows, columns=['Headword', 'POS', 'Definition'])

north = {sheet_name: align_and_split_pos(df) for sheet_name, df in north.items()}
isan = {sheet_name: align_and_split_pos(df) for sheet_name, df in isan.items()}
south = {sheet_name: align_and_split_pos(df) for sheet_name, df in south.items()}

output_path = r"output\csv_test2"

for sheet_name, df in north.items():
    df.to_csv(f"{output_path}\\cleaned_north_{sheet_name}.csv", index=False, encoding='utf-8-sig')

for sheet_name, df in isan.items():
    df.to_csv(f"{output_path}\\cleaned_northeast_{sheet_name}.csv", index=False, encoding='utf-8-sig')

for sheet_name, df in south.items():
    df.to_csv(f"{output_path}\\cleaned_south_{sheet_name}.csv", index=False, encoding='utf-8-sig')

### south

***
1st problem -- number \๑ \๒ \๓, remove "." after each row and before next meaning

sample data:
ก็กแก็ก	ว. ๑. เล็ก ๆ น้อย ๆ, เหลาะแหละ (ภ.ก.). ๒. เสียงดังเบา ๆ

desired output:
ก็กแก็ก	ว. เล็ก ๆ น้อย ๆ, เหลาะแหละ (ภ.ก.)
ก็กแก็ก	ว. เสียงดังเบา ๆ

***
2nd problem -- misalignment, remove "." after "ก็เรียก" and "ก็ว่า"

sample data:
กง	น. ๑. กง, ไม้รูปโค้งที่ตั้งเป็นโครงเรือ	
	กงข้าง	น. กงเสริมของเรือ
	กงโค้ง, กงแซม, กงหนาบ	ก็เรียก.
	กงวาน	น. กงเรือที่มีรูให้น้ำถ่ายเทไปได้ตลอดท้องเรือ
		๒. ชื่อสัตว์สี่เท้าสะเทินน้ำสะเทินบก ตระกูลเดียวกับคางคก ตัวโต มีจุดแดงบริเวณหัว อาศัยตามป่าในช่วงฤดูผสมพันธุ์ต้วผู้จะเปลี่ยนสีผิวหนังเป็นสีแดง ผิวหนังมีพิษ
	กงเขา, โก้งโค้ง, ค้างคกไฟ, โคตรค้างคก, โจงโคร่ง (ส.ฎ.), ผลักโก้งโค้ง (น.ศ.)	ก็เรียก.
		๓. ชื่อพืชคลุมดินชนิดหนึ่งคล้ายว่านหางช้าง มี ๒ ชนิด คือ กงแดงกับกงขาว. ๔. ชื่อพืชน้ำ ขึ้นตามทุ่งนา หรือพื้นที่ชุ่มน้ำ. ๕. ผ้าคลุมศีรษะของชาวไทยมุสลิมใช้ในพิธีละหมาด (น.ธ.). ๖. ส่วนรอบของล้อรถหรือเกวียน เรียก "กงล้อ". ๗. ไร่ล้มลุกที่ถางเป็นป่าหย่อม ๆ ตามเนื้อที่และกั้นเป็นขอบเขตไว้ เช่น หักล้างถางกงในดงดิน (สุวรรณวงศา). ๘. ชื่อขนมชนิดหนึ่งใช้ทำบุญในวันสารทไส้ถั่วเขียวบดละเอียดผสมน้ำตาล ชุบแป้ง ปั้นเป็นรูปต่าง ๆ แล้วทอดน้ำมัน

desired output:
กง	น.	ไม้รูปโค้งที่ตั้งเป็นโครงเรือ
กงข้าง	น.	กงเสริมของเรือ, กงโค้ง, กงแซม, กงหนาบ ก็เรียก
กงวาน	น.	กงเรือที่มีรูให้น้ำถ่ายเทไปได้ตลอดท้องเรือ
กงวาน	น.	ชื่อสัตว์สี่เท้าสะเทินน้ำสะเทินบก ตระกูลเดียวกับคางคก ตัวโต มีจุดแดงบริเวณหัวอาศัยตามป่าในช่วงฤดูผสมพันธุ์ต้วผู้จะเปลี่ยนสีผิวหนังเป็นสีแดง ผิวหนังมีพิษ, กงเขา, โก้งโค้ง, ค้างคกไฟ, โคตรค้างคก, โจงโคร่ง (ส.ฎ.), ผลักโก้งโค้ง (น.ศ.) ก็เรียก
กงวาน	น.	ชื่อพืชคลุมดินชนิดหนึ่งคล้ายว่านหางช้าง มี ๒ ชนิด คือ กงแดงกับกงขาว
กงวาน	น.	ชื่อพืชน้ำ ขึ้นตามทุ่งนา หรือพื้นที่ชุ่มน้ำ
กงวาน	น.	ผ้าคลุมศีรษะของชาวไทยมุสลิมใช้ในพิธีละหมาด (น.ธ.)
กงวาน	น.	ส่วนรอบของล้อรถหรือเกวียน เรียก "กงล้อ"
กงวาน	น.	ไร่ล้มลุกที่ถางเป็นป่าหย่อม ๆ ตามเนื้อที่และกั้นเป็นขอบเขตไว้ เช่น หักล้างถางกงในดงดิน (สุวรรณวงศา)
กงวาน	น.	ชื่อขนมชนิดหนึ่งใช้ทำบุญในวันสารทไส้ถั่วเขียวบดละเอียดผสมน้ำตาล ชุบแป้ง ปั้นเป็นรูปต่าง ๆ แล้วทอดน้ำมัน

### isan

while south use "ก็เรียก" more frequently, isan use "ก็ว่า", tho both be solved at the same time

if one of these words exists in the first row, the first row should be removed = [
	"แม่คำ",
	"ลูกคำ/ ความหมาย",
	"ความหมายลูกคำ",
	"คำสื่อ",
	"สอบถามพี่ ๆ อีกที",
	"ลูกคำ",
	"คำคล้ายลูกคำ",
	"ความหมายลูกคำ คำสื่อ",
	"ตรวจสอบกับตัวเล่ม / ถามตอง"
]

notable_outliers that must be removed (I cannot name all of them and you shouldn't hard-cod either), you must think how to remove this
[
	"พจนานุกรมเขียน "รักอย่างลำเอียง.ล" จึงลบ ".ล" ออกแล้ว (น.366)",
	"(น.250) เดิมในพจนานุกรม ระหว่าง เขียนต่อ นิยามของ ระแวดระวัง จึงแยกออกมาเป็นแม่คำ และใส่ชนิดของคำ",
	"หนังสือพิมพ์ตก ความหมาย เป็น ความหาย น. ๒๐๔"
]

sample data:
กง ๑	น. สิ่งที่เป็นวงกลม หรือขอบ เช่น กงเกียน = กงเกวียน		
	กงกวัก	น. เครื่องมือสำหรับกรอด้าย, กรอไหม	
	กงแก้ว	น. ดวงชะตา	
		กงชาตา	ก็เรียก 
	กงไกว	น. สว่าน	
	กงชาตา	ดู กงแก้ว	
กง ๒	น. เขต, แดน, บริเวณ		
	กงราฐ	น. เขตเมือง, บริเวณเมือง	
	กงโลก	น. จักรวาล	
		ขงโลก	ก็เรียก

desired output:
กง	น.	สิ่งที่เป็นวงกลม หรือขอบ เช่น กงเกียน = กงเกวียน
กง	น.	เขต, แดน, บริเวณ
กงกวัก	น.	เครื่องมือสำหรับกรอด้าย, กรอไหม
กงแก้ว	น.	ดวงชะตา, กงชาตา	ก็เรียก
กงไกว	น.	สว่าน
กงราฐ	น. เขตเมือง, บริเวณเมือง
กงโลก	น. จักรวาล, ขงโลก

as you can see the "กงชาตา	ดู กงแก้ว" is removed because it's redundant
also the second "กง" is rearranged, i think you can alphabetically sort (it should be noted that i'm not expert in thai, i might've sorted this "desired output", the ideal is to A-Z, or ก-ฮ)

### north

in addition to isan's removed list [
	"แม่คำ", "ลูกคำ/ ความหมาย",	"ความหมายลูกคำ","คำสื่อ","สอบถามพี่ ๆ อีกที","ลูกคำ","คำคล้ายลูกคำ","ความหมายลูกคำ คำสื่อ",	"ตรวจสอบกับตัวเล่ม / ถามตอง"
]

north's removed list is [
	"ช่อง 1",
	"1",
	"2",
	"3",
]

reminder, if in one of these elements in this removed list exists in X row, X row must be removed

sample data:
ช่อง 1	2	3	4
กก	น. โคน อย่าง โคนต้นไม้ 		
	น. ชื่อเรียกนกเงือก 		
	น. ชื่อแม่น้ำใน จังหวัดเชียงราย		
	น.ชื่อหญ้าชนิดหนึ่งใช้ทำเลื่อ เรียกว่าเลื่อกก		
	ก. สับ บั่น ฟัน ตัด เซ่น กก เกง สำนวน. กกหางปล่อยวัด กกหัวกกหาง		

desired output:
กก	น.	โคน อย่าง โคนต้นไม้
กก	น.	ชื่อเรียกนกเงือก
กก	น.	ชื่อแม่น้ำใน จังหวัดเชียงราย
กก	น.	ชื่อหญ้าชนิดหนึ่งใช้ทำเลื่อ เรียกว่าเลื่อกก
กก	ก.	สับ บั่น ฟัน ตัด เซ่น กก เกง
กก	สำนวน.	กกหางปล่อยวัด กกหัวกกหาง

as you can see, an incorrect row is removed, misaligned data is rearranged, when "ก." and "สำนวน." exists, it should be like this